In [1]:
import os
import sys
# # sys.path.append("../eval/")
# sys.path.append("../eval/task")
os.chdir('..')
# sys.path = sys.path[:6]
# sys.path.append("/../")
print(sys.path)
import hashlib
import time
import random
import json
import openai
import transformers
import fnmatch
import datasets
import pathlib
import torch

from functools import cache
from collections import Counter
from datasets import load_dataset
from warnings import warn
from abc import abstractmethod, ABC
from eval import tasks
from eval.tasks import utils
from eval.utils import TokenizedDataset, complete_code
from eval.tasks.utils import evaluate, convert_to_nltk_rep
# from eval.generation import parallel_generations
from diskcache import Cache
from concurrent.futures import ThreadPoolExecutor


from torch.utils.data.dataloader import DataLoader
from transformers import StoppingCriteria, StoppingCriteriaList
from accelerate.utils import set_seed

from accelerate import Accelerator, DeepSpeedPlugin
from transformers import AutoModelForCausalLM, AutoTokenizer, HfArgumentParser

from eval import tasks
from eval.generation import parallel_generations
from eval.args import RunnerArguments, HFArguments, OAIArguments, GenerationArguments
# from eval.evaluator import HFEvaluator, OAIEvaluator
from eval.tasks import ALL_TASKS, TASK_REGISTRY

%load_ext autoreload
%reload_ext autoreload

transformers.logging.set_verbosity_error()
datasets.logging.set_verbosity_error()

['/home/gatum/Projects/Neurosymbolic-AI/linc2/linc_modify/notebooks', '/home/gatum/.conda/envs/linc_modify/lib/python310.zip', '/home/gatum/.conda/envs/linc_modify/lib/python3.10', '/home/gatum/.conda/envs/linc_modify/lib/python3.10/lib-dynload', '', '/home/gatum/.conda/envs/linc_modify/lib/python3.10/site-packages']


/home/gatum/.conda/envs/linc_modify/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-08 14:32:01,202] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
data = {
   "image_id":299207,
   "question_id":"22MexNkBPpdZGX6sxbxVBH",
   "question":"What is the man by the bags awaiting?",
   "choices":[
      "skateboarder",
      "train",
      "delivery",
      "cab"
   ],
   "statement_choices":[
      "Skateboarder is the man by the bags awaiting.",
      "Train is the man by the bags awaiting.",
      "Delivery is the man by the bags awaiting.",
      "Cab is the man by the bags awaiting."
   ],
   "correct_choice_idx":3,
   "caption":"a man holding a red bag and two suitcases",
   "im_ctxs":[
      
   ],
   "ex_ctxs":[
      [
         {
            "title":"Skateboarding",
            "text":"Skateboarding is an action sport that involves riding and performing tricks using a skateboard, as well as a recreational activity, an art form, an entertainment industry job, and a method of transportation. Originating in the United States, skateboarding has been shaped and influenced by many skateboarders throughout the years. A 2009 report found that the skateboarding market is worth an estimated $4.8 billion in annual revenue, with 11.08 million active skateboarders in the world. In 2016, it was announced that skateboarding would be represented at the 2020 Summer Olympics in Tokyo, for both male and female teams.\nSince the 1970s, skateparks have been constructed specifically for use by skateboarders, freestyle BMXers, aggressive skaters, and more recently, scooters. However, skateboarding has become controversial in areas in which the activity, although legal, has damaged curbs, stoneworks, steps, benches, plazas, and parks."
         },
         {
            "title":"Skateboarder (magazine)",
            "text":"Skateboarder was primarily a digital skateboarding publication that produces a limited run of hard copy versions that are sold in skateboard shops. The publication was the United States' first skateboarding magazine. in August 2013, its editor was Jaime Owens and its publisher was Jamey Stone. On August 19, 2013, the magazine's owner GrindMedia announced that the publication would cease production on October 15, 2013.\n\n"
         },
         {
            "title":"Skateboard",
            "text":"A skateboard is a type of sports equipment used for skateboarding. It is usually made of a specially designed 7–8-ply maple plywood deck and has polyurethane wheels attached to the underside by a pair of skateboarding trucks.\nThe skateboard moves by pushing with one foot while the other foot remains balanced on the board, or by pumping one\\'s legs in structures such as a bowl or half pipe. A skateboard can also be used by standing on the deck while on a downward slope and allowing gravity to propel the board and the rider. If the rider\\'s leading foot is their left foot, they are said to ride \"regular\". Conversely, they are said to ride \"goofy\" if their leading foot is their right foot. \nThe two main types of skateboards are the longboard and shortboard. The shape of the board is also important: the skateboard must be concaved to perform tricks."
         },
         {
            "title":"Train",
            "text":"A train (from Old French trahiner, from Latin trahere, \"to pull, to draw\") is a series of connected vehicles that run along a railway track and transport people or freight. Trains are typically pulled or pushed by locomotives or railcars (often known simply as \"engines\"), though some are self-propelled, such as multiple units. Passengers and cargo are carried in railroad cars, also known as wagons. Trains are designed to a certain gauge, or distance between rails. Most trains operate on steel tracks with steel wheels, the low friction of which makes them more efficient than other forms of transport.\nTrains have their roots in wagonways, which used railway tracks and were powered by horses or pulled by cables. Following the invention of the steam locomotive in the United Kingdom in 1802, trains rapidly spread around the world, allowing freight and passengers to move over land faster and cheaper than ever possible before. Rapid transit and trams were first built in the late 1800s to transport large numbers of people in and around cities. Beginning in the 1920s, and accelerating following World War II, diesel and electric locomotives replaced steam as the means of motive power. Following the development of cars, trucks, and extensive networks of highways which offered greater mobility, as well as faster airplanes, trains declined in importance and market share, and many rail lines were abandoned. The spread of buses led to the closure of many rapid transit and tram systems during this time as well.\nSince the 1970s, governments, environmentalists, and train advocates have promoted increased use of trains due to their greater fuel efficiency and lower greenhouse gas emissions compared to other modes of land transport. High-speed rail, first built in the 1960s, has proven competitive with cars and planes over short to medium distances. Commuter rail has grown in importance since the 1970s as an alternative to congested highways and a means to promote development, as has light rail in the 21st century. Freight trains remain important for the transport of bulk commodities such as coal and grain, as well as being a means of reducing road traffic congestion by freight trucks.\nWhile conventional trains operate on relatively flat tracks with two rails, a number of specialized trains exist which are significantly different in their mode of operation. Monorails operate on a single rail, while funiculars and rack railways are uniquely designed to traverse steep slopes. Experimental trains such as high speed maglevs, which use magnetic levitation to float above a guideway, are under development in the 2020s and offer higher speeds than even the fastest conventional trains. Trains which use alternative fuels such as natural gas and hydrogen are another 21st-century development."
         },
         {
            "title":"Delivery Hero",
            "text":"Delivery Hero SE is a German multinational online food ordering and food delivery company based in Berlin, Germany. Founded in 2011, the company operates in 70+ countries internationally in Europe, Asia, Africa, Latin and South America, and the Middle East, and partners with 500,000+ restaurants. Delivery Hero has increasingly branched out beyond food delivery, and is a leading player in the emerging category of quick commerce, which delivers small batch orders in under an hour.\nIn the third quarter of 2021, Delivery Hero processed 791 million orders—that equals a year-on-year growth of 52%.\nWhile Delivery Hero is headquartered in Germany with offices worldwide, as a \"gig economy company\" (akin to Meituan, Uber or Airbnb) nearly all of the company\\'s deliveries are carried out by workers using motorcycles, bicycles and cars, dispatched via the company\\'s smartphone apps.\nIn general, and in contrast to recent legal precedents in Canada and Australia, Delivery Hero does not classify these couriers as employees. This policy has led to ongoing legal battles and labour disputes, and may be linked to the shutdowns of Delivery Hero\\'s operations in several countries."
         },
         {
            "title":"Delivery (cricket)",
            "text":"A delivery or ball in cricket is a single action of bowling a cricket ball toward the batter. Once the ball has been delivered, batters may attempt to score runs, with the bowler and other fielders attempting to stop this by getting the batters out. When the ball becomes dead, the next delivery can begin.\nDuring the play of the game, a member of the fielding team is designated as the bowler and bowls deliveries toward the batter. Six legal balls in a row constitutes an over, after which a different member of the fielding side takes over the role of bowler for the next over. The bowler delivers the ball from their end of the pitch toward the batter standing at the opposite wicket at the other end of the pitch. Bowlers can be either left-handed or right-handed. This approach to their delivery, in addition to their decision of bowling around the wicket (from the sides of the wicket on the bowler\\'s end) or over the wicket, is knowledge of which the umpire and the batter are to be made aware.\nSome forms of cricket limit the number of legal deliveries that may be bowled in each innings; for example, 100-ball cricket has a maximum of 200 legal deliveries in the game (unless there is a tie). Every delivery in a cricket match is notated by the number of completed overs that have occurred before that delivery, as well as which delivery in the current over that delivery is; for example, \"Over 14.2\" indicates that 14 overs have been completed and the second delivery of the 15th over is the one under consideration. Since illegal deliveries do not count towards the progression of an over, a set of consecutive illegal deliveries (and the legal delivery that follows them) will have the same notation. Illegal deliveries generally occur when a bowler bowls from too close to the batter, or delivers the ball out of the batter\\'s reach.\n\n"
         },
         {
            "title":"Cab Calloway",
            "text":"Cabell Calloway III (December 25, 1907 – November 18, 1994) was an American jazz singer and bandleader. He was a regular performer at the Cotton Club in Harlem, where he became a popular vocalist of the swing era. His niche of mixing jazz and vaudeville won him acclaim during a career that spanned over 65 years.\nCalloway was a master of energetic scat singing and led one of the most popular dance bands in the United States from the early 1930s to the late 1940s. His band included trumpeters Dizzy Gillespie, Jonah Jones, and Adolphus \"Doc\" Cheatham, saxophonists Ben Webster and Leon \"Chu\" Berry, guitarist Danny Barker, bassist Milt Hinton, and drummer Cozy Cole.\nCalloway had several hit records in the 1930s and 1940s, becoming the first African-American musician to sell one million copies of a single record. He became known as the \"Hi-de-ho\" man of jazz for his most famous song, \"Minnie the Moocher\", originally recorded in 1931. He reached the Billboard charts in five consecutive decades (1930s–1970s). Calloway also made several stage, film, and television appearances until his death in 1994 at the age of 86. He had roles in Stormy Weather (1943), Porgy and Bess (1953), The Cincinnati Kid (1965), and Hello Dolly! (1967). His career enjoyed a marked resurgence from his appearance in the 1980 film The Blues Brothers.\nCalloway was the first African-American to have a nationally syndicated radio program. In 1993, Calloway received the National Medal of Arts from the United States Congress. He posthumously received the Grammy Lifetime Achievement Award in 2008. His song \"Minnie the Moocher\" was inducted into the Grammy Hall of Fame in 1999, and added to the Library of Congress\\' National Recording Registry in 2019. Three years later in 2022, the National Film Registry selected his home films for preservation as \"culturally, historically or aesthetically significant films\". He is also inducted into the Big Band and Jazz Hall of Fame and the International Jazz Hall of Fame.\n\n"
         },
         {
            "title":"Death Cab for Cutie",
            "text":"Death Cab for Cutie (commonly abbreviated to DCFC or Death Cab) is an American rock band formed in Bellingham, Washington, in 1997. Death Cab for Cutie's music has been classified as indie rock, indie pop, and alternative rock. The band is currently composed of Ben Gibbard (vocals, guitar, piano), Nick Harmer (bass), Dave Depper (guitar, keyboards, backing vocals), Zac Rae (keyboards, guitar), and Jason McGerr (drums). \nThe band was originally a solo project by Gibbard, who expanded the project into a complete group upon getting a record deal. They released their debut album, Something About Airplanes, in 1998. The band's fourth album, 2003's Transatlanticism, broke into the mainstream both critically and commercially; its songs were featured in various TV series and films. The band's major label debut for Atlantic Records, 2005's Plans, went platinum. Founding guitarist Chris Walla left the band in 2014 after 17 years. The band's tenth and latest studio album, Asphalt Meadows, was released on September 16, 2022. Alongside their ten full-length studio albums, the band has released four EPs, two live EPs, one live album, and one demo album.\n\n"
         }
      ]
   ]
}

In [19]:
lines = ''
for ex_ctx in data['ex_ctxs'][0]:
    lines += ex_ctx['text'].strip() + "\n"
    print(ex_ctx['text'].strip())
    print('==')

Skateboarding is an action sport that involves riding and performing tricks using a skateboard, as well as a recreational activity, an art form, an entertainment industry job, and a method of transportation. Originating in the United States, skateboarding has been shaped and influenced by many skateboarders throughout the years. A 2009 report found that the skateboarding market is worth an estimated $4.8 billion in annual revenue, with 11.08 million active skateboarders in the world. In 2016, it was announced that skateboarding would be represented at the 2020 Summer Olympics in Tokyo, for both male and female teams.
Since the 1970s, skateparks have been constructed specifically for use by skateboarders, freestyle BMXers, aggressive skaters, and more recently, scooters. However, skateboarding has become controversial in areas in which the activity, although legal, has damaged curbs, stoneworks, steps, benches, plazas, and parks.
==
Skateboarder was primarily a digital skateboarding pub

In [20]:
print(lines)

Skateboarding is an action sport that involves riding and performing tricks using a skateboard, as well as a recreational activity, an art form, an entertainment industry job, and a method of transportation. Originating in the United States, skateboarding has been shaped and influenced by many skateboarders throughout the years. A 2009 report found that the skateboarding market is worth an estimated $4.8 billion in annual revenue, with 11.08 million active skateboarders in the world. In 2016, it was announced that skateboarding would be represented at the 2020 Summer Olympics in Tokyo, for both male and female teams.
Since the 1970s, skateparks have been constructed specifically for use by skateboarders, freestyle BMXers, aggressive skaters, and more recently, scooters. However, skateboarding has become controversial in areas in which the activity, although legal, has damaged curbs, stoneworks, steps, benches, plazas, and parks.
Skateboarder was primarily a digital skateboarding public

In [ ]:
# The following is a first-order logic (FOL) problem.
# The problem is to determine whether the conclusion follows from the premises.
# The premises are given in the form of a set of first-order logic sentences.
# The conclusion is given in the form of a single first-order logic sentence.
# The task is to translate each of the premises and conclusions into FOL expressions, so that the expressions can be evaluated by a theorem solver to determine whether the conclusion follows from the premises.
# Expressions should be adhere to the format of the Python NLTK package logic module.


# <PREMISES>
# All dispensable things are environment-friendly.
# All woodware is dispensable.
# All paper is woodware.
# No good things are bad.
# All environment-friendly things are good.
# A worksheet is either paper or is environment-friendly.
# </PREMISES>
# <CONCLUSION>
# A worksheet is not dispensable.
# </CONCLUSION>
# <EVALUATE>
# TEXT:	All dispensable things are environment-friendly.
# FOL:	all x. (Dispensable(x) -> EnvironmentFriendly(x))
# TEXT:	All woodware is dispensable.
# FOL:	all x. (Woodware(x) -> Dispensable(x))
# TEXT:	All paper is woodware.
# FOL:	all x. (Paper(x) -> Woodware(x))
# TEXT:	No good things are bad.
# FOL:	all x. (Good(x) -> -Bad(x))
# TEXT:	All environment-friendly things are good.
# FOL:	all x. (EnvironmentFriendly(x) -> Good(x))
# TEXT:	A worksheet is either paper or is environment-friendly.
# FOL:	((Paper(Worksheet) & -EnvironmentFriendly(Worksheet)) | (-Paper(Worksheet) & EnvironmentFriendly(Worksheet)))
# TEXT:	A worksheet is not dispensable.
# FOL:	-Dispensable(Worksheet)
# </EVALUATE>

# <PREMISES>
# A La Liga soccer team ranks higher than another if it receives more points.
# If two La Liga soccer teams recieve the same points, the team which recieves more points from the games between the two teams ranks higher.
# Real Madrid and Barcelona are both La Liga soccer teams.
# In La Liga 2021-2022, Real Madrid recieves 86 points and Barcelon recieves 73 points.
# In La Liga 2021-2022, Real Madrid and Barcelona both recieve 3 points from the games between them.
# </PREMISES>
# <CONCLUSION>
# In La Liga 2021-2022, Real Madrid ranks higher than Barcelona.
# </CONCLUSION>
# <EVALUATE>
# TEXT:	A La Liga soccer team ranks higher than another if it receives more points.
# FOL:	all x. all y. (LaLiga(x) & LaLiga(y) & MorePoints(x, y) -> HigherRank(x, y))
# TEXT:	If two La Liga soccer teams recieve the same points, the team which recieves more points from the games between the two teams ranks higher.
# FOL:	all x. all y. (LaLiga(x) & LaLiga(y) & -MorePoints(x, y) & -MorePoints(y, x) & MorePointsInGameBetween(x, y) -> HigherRank(x, y))
# TEXT:	Real Madrid and Barcelona are both La Liga soccer teams.
# FOL:	LaLiga(RealMadrid) & LaLiga(Barcelona)
# TEXT:	In La Liga 2021-2022, Real Madrid recieves 86 points and Barcelon recieves 73 points.
# FOL:	MorePoints(RealMadrid, Barcelona)
# TEXT:	In La Liga 2021-2022, Real Madrid and Barcelona both recieve 3 points from the games between them.
# FOL:	-MorePointsInGameBetween(RealMadrid, Barcelona) & -MorePointsInGameBetween(Barcelona, RealMadrid)
# TEXT:	In La Liga 2021-2022, Real Madrid ranks higher than Barcelona.
# FOL:	HigherRank(RealMadrid, Barcelona)
# </EVALUATE>

# <PREMISES>
# All athletes are good at sports.
# All Olympic gold medal winners are good athletes.
# No scientists are good at sports.
# All Nobel laureates are scientists.
# Amy is good at sports or Amy is an Olympic gold medal winner.
# If Amy is not a Nobel laureate, then Amy is not an Olympic gold medal winner.
# </PREMISES>
# <CONCLUSION>
# If Amy is not an Olympic gold medal winner, then Amy is a Nobel laureate.
# </CONCLUSION>
# <EVALUATE>
# TEXT:	All athletes are good at sports.
# FOL:	all x. (Athlete(x) -> GoodAtSports(x))
# TEXT:	All Olympic gold medal winners are good athletes.
# FOL:	all x. (OlympicGoldMedalWinner(x) -> Athlete(x))
# TEXT:	No scientists are good at sports.
# FOL:	all x. (Scientist(x) -> -GoodAtSports(x))
# TEXT:	All Nobel laureates are scientists.
# FOL:	all x. (NobelLaureate(x) -> Scientist(x))
# TEXT:	Amy is good at sports or Amy is an Olympic gold medal winner.
# FOL:	GoodAtSports(Amy) | OlympicGoldMedalWinner(Amy)
# TEXT:	If Amy is not a Nobel laureate, then Amy is not an Olympic gold medal winner.
# FOL:	-NobelLaureate(Amy) -> -OlympicGoldMedalWinner(Amy)
# TEXT:	If Amy is not an Olympic gold medal winner, then Amy is a Nobel laureate.
# FOL:	-OlympicGoldMedalWinner(Amy) -> NobelLaureate(Amy)
# </EVALUATE>
